In [1]:
# Mount to Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Define Project Folder
FOLDERNAME = 'Colab\ Notebooks/SC201L16'

%cd drive/MyDrive/$FOLDERNAME

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/SC201L16


In [2]:
              # Define device
import torch
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')
print('Device:', device)

Device: cuda


In [3]:
# Load Existing Dataset
import torchvision.datasets as dset
import torchvision.transforms as T
transform = T.Compose([T.ToTensor(),T.RandomAutocontrast()])
train_data = dset.MNIST('./train',train=True,download=True,transform=transform)
val_data = dset.MNIST('./val',train=False,download=True,transform=transform)

In [4]:
# Check Data Dimension
print(train_data)

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./train
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               RandomAutocontrast(p=0.5)
           )


In [5]:

train_data[0][0].shape ## print data

torch.Size([1, 28, 28])

In [6]:
num_train = len(train_data)
num_val = len(val_data)
print('Number of training:', num_train)
print('Number of validation:', num_val)

Number of training: 60000
Number of validation: 10000


In [7]:
# Build Model
import torch.nn as nn
model = None
# model = nn.Sequential(
#     # N x 1 x 28 x 28

#     nn.Conv2d(in_channels=1,out_channels=64,kernel_size=3,padding=1,stride =1),
#     nn.BatchNorm2d(num_features=64),
#     nn.ReLU(),
#     nn.MaxPool2d(kernel_size=2,stride=2),
#     # 16 x 64 x 14 x 14

#     nn.Flatten(),
#     # 10 neurons(0-9)
#     nn.Linear(64*14*14,10)
# )
# class MyModel(nn.Module):
#   def __init__(self):
#     super().__init__()
#     self.conv2d = nn.Conv2d(1,64,3,1,1)
#     self.bn2d = nn.BatchNorm2d(64)
#     self.fc = nn.Linear(64*14*14,10)

#   def forward(self,x):
#     out = self.conv2d(x)
#     out = self.bn2d(out)
#     out = nn.functional.relu(out)
#     out = nn.functional.max_pool2d(out,(2,2))
#     out = torch.flatten(out,1)
#     out = self.fc(out)
#     return out
# model = MyModel()
class MyRNN(nn.Module):
  def __init__(self,input_size,hidden_size,output_size):
    super().__init__()
    self.lstm = nn.LSTM(input_size,hidden_size,batch_first=True)
    self.fc = nn.Linear(hidden_size,output_size)
  def forward(self,x):
    x = x.squeeze() # N x 28 x 28
    #print(x)
    output, (h_n,c_n) = self.lstm(x)
    out = output[:,-1,:]
    out= self.fc(out)
    return out

model = MyRNN(28,64,10)


In [8]:
# Move model to GPU
# -> GPU
model = model.cuda()

In [9]:
# Create Mini-batches
from torch.utils.data import DataLoader
mini_trains = DataLoader(train_data, batch_size=128, shuffle=True)
mini_vals = DataLoader(val_data, batch_size=128, shuffle=True)

In [10]:
# Training Procedure
def train(num_epoch, model, mini_trains, mini_vals, device, loss_function, optimizer):
  for epoch in range(num_epoch):
    num_iters = 0
    for x, y in mini_trains:
      model.train()
      x = x.to(device)
      y = y.to(device)
      scores = model(x)
      loss = loss_function(scores, y)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      if num_iters % 10 == 0:
        evaluate_predictor(model, epoch, mini_vals, device)
      num_iters += 1

In [11]:
# Validating Procedure
def evaluate_predictor(model, epoch, mini_vals, device):
  model.eval()
  with torch.no_grad():
    acc_count = 0
    for x, y in mini_vals:
      x = x.to(device)
      y = y.to(device)
      scores=model(x)
      predictions=scores.max(1)[1]
      acc = predictions.eq(y).sum().item()
      acc_count += acc
    print(f'Epoch[{epoch+1}] Acc: {acc_count/num_val}')

In [12]:
# Define loss function & optimizer
import torch.optim as optim
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [13]:
# Start training
train(1, model, mini_trains, mini_vals, device, loss_function, optimizer)

Epoch[1] Acc: 0.0945
Epoch[1] Acc: 0.1009
Epoch[1] Acc: 0.1105
Epoch[1] Acc: 0.2014
Epoch[1] Acc: 0.2485
Epoch[1] Acc: 0.3501
Epoch[1] Acc: 0.5235
Epoch[1] Acc: 0.6016
Epoch[1] Acc: 0.636
Epoch[1] Acc: 0.6537
Epoch[1] Acc: 0.6968
Epoch[1] Acc: 0.6912
Epoch[1] Acc: 0.7249
Epoch[1] Acc: 0.7025
Epoch[1] Acc: 0.765
Epoch[1] Acc: 0.7593
Epoch[1] Acc: 0.7845
Epoch[1] Acc: 0.7905
Epoch[1] Acc: 0.7881
Epoch[1] Acc: 0.8178
Epoch[1] Acc: 0.8305
Epoch[1] Acc: 0.8405
Epoch[1] Acc: 0.8477
Epoch[1] Acc: 0.8539
Epoch[1] Acc: 0.8583
Epoch[1] Acc: 0.8608
Epoch[1] Acc: 0.8604
Epoch[1] Acc: 0.8754
Epoch[1] Acc: 0.8851
Epoch[1] Acc: 0.8824
Epoch[1] Acc: 0.8869
Epoch[1] Acc: 0.8851
Epoch[1] Acc: 0.8839
Epoch[1] Acc: 0.894
Epoch[1] Acc: 0.8869
Epoch[1] Acc: 0.8963
Epoch[1] Acc: 0.8968
Epoch[1] Acc: 0.907
Epoch[1] Acc: 0.8916
Epoch[1] Acc: 0.904
Epoch[1] Acc: 0.9133
Epoch[1] Acc: 0.9114
Epoch[1] Acc: 0.919
Epoch[1] Acc: 0.9176
Epoch[1] Acc: 0.9159
Epoch[1] Acc: 0.9121
Epoch[1] Acc: 0.9152
